In [1]:
import numpy as np
import pandas as pd
import string
import re
import gensim
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from impala.dbapi import connect

In [2]:
connection = connect(
host='172.26.88.8',
port=21050,
user=user,
password=password,
auth_mechanism='PLAIN',
use_ssl=True,
database='default'
)

In [4]:
#getting the df for a given postal code
sql = '''
    SELECT CAST(account AS BIGINT) AS account, uid, hh_size, address_1, address_2, address_3, address_postal_code, hh_gps_gnd
    FROM adl_db.hhv_2_2_july_connection_hhs 
    WHERE address_postal_code = '90015'
    ORDER BY account
     '''
def create_data_from_sql(sql):
    df = pd.read_sql(
        sql,
        con=connection)
    return df

df = create_data_from_sql(sql)
df.head()

,account,uid,hh_size,address_1,address_2,address_3,address_postal_code,hh_gps_gnd
0,60102822,23852,6,SAMANTHA TEA ROOM,'N',MEEGAHAKIULA,90015,None
1,60128019,354111,14,22 LIHINI MW,'N',MEEGAHAKIULA,90015,akurukaduwa
2,60276041,2944276,11,"KIWLA ROAD,","NIKAPOTHA,",MEEGAHAKIULA.,90015,None
3,60305581,411740,23,18 MILE POST,BALAGOLLA,MEEGAHAKIWULA,90015,balagolla
4,65781816,277867,12,19TH MILE POST,KARAMITIYA,MEEGAHAKIWLA,90015,atigala east


In [5]:
len(df)

150

In [6]:
postal_code = df['address_postal_code'][0]

In [7]:
def process_lines(line):
  line = line.strip() if not (line is np.nan) else ''
  line = line if len(line) > 1 else ''
  line = line.strip(string.punctuation)

  return line

def process_final_address(row):
  line1 = row['address_1']
  line2 = row['address_2']
  line3 = row['address_3']

  line1 = process_lines(line1)
  line2 = process_lines(line2)
  line3 = process_lines(line3)

  address = '{} {} {}'.format(line1,line2,line3)
  return address.strip()

In [8]:
#preprocessing the data
df = df.apply(lambda x: x.astype(str).str.lower())
df = df.apply(lambda x: x.astype(str).str.strip())
df = df.apply(lambda x: x.astype(str).str.replace('\'n\'', ''))
df = df.apply(lambda x: x.astype(str).str.replace('nan', ''))

df['address'] = df.apply(process_final_address, axis=1)

df['address']= [re.sub(r"[,.;:/*-]+\ *", ' ', x) for x in df['address'].tolist()]
df['address']=df['address'].replace(('"', ' '),regex=True)
df = df.apply(lambda x: x.astype(str).str.replace('no ', ''))
df = df.apply(lambda x: x.astype(str).str.replace(' null', ''))
df['address']=df['address'].replace({' +':' '},regex=True)

df['uid'] = df['uid'].astype(int)

In [9]:
#lemmatization
df['address']= [re.sub('( mawatha| road| para| mw| rd| lane| mawetha)',' road', x) for x in df['address'].tolist()]
df['address']= [re.sub('( avenue| av)',' avenue', y) for y in df['address'].tolist()]

In [10]:
#tokenization of the address column in the df 
def tokenization(address):
    address_tokens = [x.split(" ") for x in address]
    return address_tokens 

df['address_tokens']=tokenization(df['address'])

In [11]:
# getting the mode of the address tokens lengths
x=[]
for i in range(len(df)):
    x.append(len(df['address_tokens'][i]))
    
from statistics import mode
mode=mode(x)

In [12]:
#padding the address tokens
maxlength=mode
pad_token='pad'
def padding(address_tokens):
    if len(address_tokens) >= maxlength:
        pad_address_tokens = address_tokens[-maxlength:]
    else:
        pad_address_tokens = [pad_token]*(maxlength-len(address_tokens))+ address_tokens
    return pad_address_tokens

In [13]:
#getting the padding address tokens to a list
def pad_address_tokens(address_tokens):
    pad_address_tokens=[]
    for i in address_tokens:
        pad_address_tokens.append(padding(i))
    return pad_address_tokens

df['pad_address_tokens']=pad_address_tokens(df['address_tokens'])

In [14]:
#making a flat list from the pad address tokens
def flat_list(pad_address_tokens):
    flat_list=[item for sublist in pad_address_tokens for item in sublist]
    return flat_list

flat_list = flat_list(df['pad_address_tokens'].tolist())

In [15]:
#creating a pad address corpus
def pad_address_corpus(flat_list):
    corpus=' '.join(flat_list)
    return corpus

corpus = pad_address_corpus(flat_list)

In [16]:
#creating embedding model
def wordEmbedding(corpus,modelType,size):
    model = Word2Vec(df['pad_address_tokens'], size=size, window=10,workers=4,min_count=1,sg=modelType)
    return model

model = wordEmbedding(corpus,1,100)

In [17]:
# to create a dictionary object for the vocabulary
def my_dict(model):
    my_dict = {key:model.wv[key] for key in model.wv.vocab}
    return my_dict

my_dict = my_dict(model)

In [18]:
#creating (12,100) matrices for all the addresses
def pad_address_matrix_for_all_sentences(pad_address_tokens):
    pad_address_matrix_for_all_sentences=[]
    for i in range(len(pad_address_tokens)):
        pad_address_matrix=[]
        for j in range(len(pad_address_tokens[i])):
            pad_address_matrix.append(my_dict[pad_address_tokens[i][j]])
        pad_address_matrix_for_all_sentences.append(pad_address_matrix)
    pad_address_matrix_for_all_sentences = np.array(pad_address_matrix_for_all_sentences)
    return pad_address_matrix_for_all_sentences

pad_address_matrix_for_all_sentences=pad_address_matrix_for_all_sentences(df['pad_address_tokens'])
df['pad_address_matrix']=pad_address_matrix_for_all_sentences.tolist()

In [19]:
df.head()

,account,uid,hh_size,address_1,address_2,address_3,address_postal_code,hh_gps_gnd,address,address_tokens,pad_address_tokens,pad_address_matrix
0,60102822,23852,6,samantha tea room,,meegahakiula,90015,,samantha tea room meegahakiula,"[samantha, tea, room, meegahakiula]","[pad, pad, samantha, tea, room, meegahakiula]","[[-0.0035762805491685867, 0.00479148980230093,..."
1,60128019,354111,14,22 lihini mw,,meegahakiula,90015,akurukaduwa,22 lihini road meegahakiula,"[22, lihini, road, meegahakiula]","[pad, pad, 22, lihini, road, meegahakiula]","[[-0.0035762805491685867, 0.00479148980230093,..."
2,60276041,2944276,11,"kiwla road,","nikapotha,",meegahakiula.,90015,,kiwla road nikapotha meegahakiula,"[kiwla, road, nikapotha, meegahakiula]","[pad, pad, kiwla, road, nikapotha, meegahakiula]","[[-0.0035762805491685867, 0.00479148980230093,..."
3,60305581,411740,23,18 mile post,balagolla,meegahakiwula,90015,balagolla,18 mile post balagolla meegahakiwula,"[18, mile, post, balagolla, meegahakiwula]","[pad, 18, mile, post, balagolla, meegahakiwula]","[[-0.0035762805491685867, 0.00479148980230093,..."
4,65781816,277867,12,19th mile post,karamitiya,meegahakiwla,90015,atigala east,19th mile post karamitiya meegahakiwla,"[19th, mile, post, karamitiya, meegahakiwla]","[pad, 19th, mile, post, karamitiya, meegahakiwla]","[[-0.0035762805491685867, 0.00479148980230093,..."


In [20]:
#creating the connection edge df

s=[]
for i in range(len(df)):
    l=[]
    for j in range(len(df)):
        if i<j:
            l.append([df.index[i],df['account'][df.index[i]],df.index[j],df['account'][df.index[j]]])
    s.append(l)
            
df1=pd.DataFrame(s[0],columns=['index 1','account 1','index 2','account 2'])

for i in range(1,len(df)):
    df1 = df1.append(pd.DataFrame(s[i],columns=['index 1','account 1','index 2','account 2']))
df1

,index 1,account 1,index 2,account 2
0,0,60102822,1,60128019
1,0,60102822,2,60276041
2,0,60102822,3,60305581
3,0,60102822,4,65781816
4,0,60102822,5,65824736
...,...,...,...,...
1,146,778899014,148,814266317
2,146,778899014,149,914219347
0,147,779363343,148,814266317
1,147,779363343,149,914219347


In [21]:
len(df1)

11175

In [22]:
#adding the columns to the connection edge df
df1=df1.set_index('index 1')
df1['postal code']=postal_code
df1['gnd 1']=df['hh_gps_gnd'][df1.index]
df1['uid 1']=df['uid'][df1.index]
df1['address 1']=df['address'][df1.index]
df1['pad_address_matrix 1']=df['pad_address_matrix'][df1.index]

df1=df1.reset_index()
df1=df1.set_index('index 2')
df1['gnd 2']=df['hh_gps_gnd'][df1.index]
df1['uid 2']=df['uid'][df1.index]
df1['address 2']=df['address'][df1.index]
df1['pad_address_matrix 2']=df['pad_address_matrix'][df1.index]
df1=df1.reset_index()
df1.head()

,index 2,index 1,account 1,account 2,postal code,gnd 1,uid 1,address 1,pad_address_matrix 1,gnd 2,uid 2,address 2,pad_address_matrix 2
0,1,0,60102822,60128019,90015,,23852,samantha tea room meegahakiula,"[[-0.0035762805491685867, 0.00479148980230093,...",akurukaduwa,354111,22 lihini road meegahakiula,"[[-0.0035762805491685867, 0.00479148980230093,..."
1,2,0,60102822,60276041,90015,,23852,samantha tea room meegahakiula,"[[-0.0035762805491685867, 0.00479148980230093,...",,2944276,kiwla road nikapotha meegahakiula,"[[-0.0035762805491685867, 0.00479148980230093,..."
2,3,0,60102822,60305581,90015,,23852,samantha tea room meegahakiula,"[[-0.0035762805491685867, 0.00479148980230093,...",balagolla,411740,18 mile post balagolla meegahakiwula,"[[-0.0035762805491685867, 0.00479148980230093,..."
3,4,0,60102822,65781816,90015,,23852,samantha tea room meegahakiula,"[[-0.0035762805491685867, 0.00479148980230093,...",atigala east,277867,19th mile post karamitiya meegahakiwla,"[[-0.0035762805491685867, 0.00479148980230093,..."
4,5,0,60102822,65824736,90015,,23852,samantha tea room meegahakiula,"[[-0.0035762805491685867, 0.00479148980230093,...",,2488127,eam maliban textiles pvt ltd 19th milepost kar...,"[[-0.004310681018978357, 0.0002084777224808931..."


In [23]:
#Getting similarity of addresses between connections
def similarity(row):
    x=cosine_similarity(np.array(row['pad_address_matrix 1']).reshape(1,-1),np.array(row['pad_address_matrix 2']).reshape(1,-1)).squeeze()
    return x

df1['similarity'] = df1.apply(similarity, axis=1)
df1['similarity'] = df1['similarity'].astype(float)

df1.set_index(['index 1', 'index 2'], inplace=True)
df1 = df1[['postal code','gnd 1','gnd 2','uid 1','uid 2','account 1', 'account 2','address 1','address 2','similarity']]
df1.head()

postal code gnd 1         gnd 2  uid 1    uid 2 account 1  \
index 1 index 2                                                             
0       1             90015         akurukaduwa  23852   354111  60102822   
        2             90015                      23852  2944276  60102822   
        3             90015           balagolla  23852   411740  60102822   
        4             90015        atigala east  23852   277867  60102822   
        5             90015                      23852  2488127  60102822   

                account 2                       address 1  \
index 1 index 2                                             
0       1        60128019  samantha tea room meegahakiula   
        2        60276041  samantha tea room meegahakiula   
        3        60305581  samantha tea room meegahakiula   
        4        65781816  samantha tea room meegahakiula   
        5        65824736  samantha tea room meegahakiula   

                                                         address 2  similarity  
index 1 index 2                                                                 
0       1                              22 lihini road meegahakiula    0.609716  
        2                        kiwla road nikapotha meegahakiula    0.623269  
        3                     18 mile post balagolla meegahakiwula    0.230906  
        4                   19th mile post karamitiya meegahakiwla    0.290980  
        5        eam maliban textiles pvt ltd 19th milepost kar...    0.238994

In [24]:
#sorting according to the similarity
df1.sort_values('similarity',ascending=False)

postal code          gnd 1          gnd 2    uid 1    uid 2  \
index 1 index 2                                                               
91      145           90015  aggalaulpatha     emunugalla  2694538  2234312   
61      91            90015  aggalaulpatha  aggalaulpatha  2694538  2694538   
        145           90015  aggalaulpatha     emunugalla  2694538  2234312   
12      129           90015       dankanda       dankanda   348402   348402   
72      139           90015                                1289144  1289144   
...                     ...            ...            ...      ...      ...   
14      58            90015                     balagolla   829727   411740   
11      129           90015          arawa       dankanda   629227   348402   
        12            90015          arawa       dankanda   629227   348402   
126     133           90015  meegahakivula                  254682   387256   
11      111           90015          arawa   badulla east   629227   672121   

                 account 1  account 2  \
index 1 index 2                         
91      145      769704461  778890068   
61      91       762664461  769704461   
        145      762664461  778890068   
12      129       80992042  772641858   
72      139      766195196  777190693   
...                    ...        ...   
14      58        81156155  761789091   
11      129       80972263  772641858   
        12        80972263   80992042   
126     133      772262293  773552135   
11      111       80972263  769905527   

                                                         address 1  \
index 1 index 2                                                      
91      145                16th mile post arawa road meegahakivula   
61      91                 16th mile post arawa road meegahakivula   
        145                16th mile post arawa road meegahakivula   
12      129                  nadun niwasa akurukaduwa neegahakiula   
72      139                                 isurusiri meegahkiwula   
...                                                            ...   
14      58       dedunu sewana welanlanda junction akurukaduwa ...   
11      129        153 2 aluthyaya kirigallanda arawa meegahakiula   
        12         153 2 aluthyaya kirigallanda arawa meegahakiula   
126     133                            26 1 kanugolla meegahakiula   
11      111        153 2 aluthyaya kirigallanda arawa meegahakiula   

                                                         address 2  similarity  
index 1 index 2                                                                 
91      145                16th mile post arawa road meegahakivula    1.000000  
61      91                 16th mile post arawa road meegahakivula    1.000000  
        145                16th mile post arawa road meegahakivula    1.000000  
12      129                  nadun niwasa akurukaduwa neegahakiula    1.000000  
72      139                                 isurusiri meegahkiwula    1.000000  
...                                                            ...         ...  
14      58                    18 mile post balagolla meegahakiwula   -0.003105  
11      129                  nadun niwasa akurukaduwa neegahakiula   -0.007302  
        12                   nadun niwasa akurukaduwa neegahakiula   -0.007302  
126     133      178 a1 dummatikanda 18th mile post balagolla m...   -0.010132  
11      111                                      c o nalaka dasena   -0.022744  

[11175 rows x 10 columns]